<div style='text-align: center;'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzJzIHdangJTrH2mFXFgsLjuLCjpfXXwbxg&usqp=CAU' width='100'/>
    <h1>Sharif University of Technology</h1>
    <h2>Natural Language Processing</h2>
    <h3>Final Project</h3>
    <h4>Spoiler classification and summary generation</h4>
    <p><strong>Authors:</strong> Parnian Razavipour, Mobina Salimipanah</p>
    <p><strong>(Equal Contribution)</strong></p>
</div>
<hr/>


In [ ]:
!pip install transformers
!pip install datasets
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install transformers datasets torch


##**Download and Load Dataset**

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d rmisra/imdb-spoiler-dataset

!unzip imdb-spoiler-dataset.zip

!ls

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/rmisra/imdb-spoiler-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
100% 330M/331M [00:20<00:00, 20.6MB/s]
100% 331M/331M [00:20<00:00, 16.7MB/s]
Archive:  imdb-spoiler-dataset.zip
  inflating: IMDB_movie_details.json  
  inflating: IMDB_reviews.json       
IMDB_movie_details.json  IMDB_reviews.json  imdb-spoiler-dataset.zip  sample_data


In [ ]:
import pandas as pd

movie_details = pd.read_json('IMDB_movie_details.json', lines=True)
reviews = pd.read_json('IMDB_reviews.json', lines=True)
print(movie_details.head())
print(reviews.head())


    movie_id                                       plot_summary  duration  \
0  tt0105112  Former CIA analyst, Jack Ryan is in England wi...  1h 57min   
1  tt1204975  Billy (Michael Douglas), Paddy (Robert De Niro...  1h 45min   
2  tt0243655  The setting is Camp Firewood, the year 1981. I...  1h 37min   
3  tt0040897  Fred C. Dobbs and Bob Curtin, both down on the...   2h 6min   
4  tt0126886  Tracy Flick is running unopposed for this year...  1h 43min   

                         genre  rating release_date  \
0           [Action, Thriller]     6.9   1992-06-05   
1                     [Comedy]     6.6   2013-11-01   
2            [Comedy, Romance]     6.7   2002-04-11   
3  [Adventure, Drama, Western]     8.3   1948-01-24   
4     [Comedy, Drama, Romance]     7.3   1999-05-07   

                                       plot_synopsis  
0  Jack Ryan (Ford) is on a "working vacation" in...  
1  Four boys around the age of 10 are friends in ...  
2                                        

In [ ]:
# Drop rows where 'plot_synopsis' or 'plot_summary' is missing
movie_details.dropna(subset=['plot_synopsis', 'plot_summary'], inplace=True)


##**training and test sets**

In [ ]:

# Split the data into training and test sets
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(movie_details, test_size=0.3, random_state=42)  # 70% for training, 30% for temp

validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)  # Split the 30% into 15% validation and 15% test


In [ ]:
train_data.shape

(1100, 7)

In [ ]:
# Example exploration: Average length of plot_synopsis
movie_details['synopsis_length'] = movie_details['plot_synopsis'].apply(lambda x: len(x.split()))
print(movie_details['synopsis_length'].describe())


count     1572.000000
mean      1439.085242
std       1496.392929
min          0.000000
25%        493.750000
50%       1073.500000
75%       1920.000000
max      11396.000000
Name: synopsis_length, dtype: float64


In [ ]:
train_data.to_json('/content/train_data.json', orient='records', lines=True)
validation_data.to_json('/content/val_data.json', orient='records', lines=True)
test_data.to_json('/content/test_data.json', orient='records', lines=True)


In [ ]:
from datasets import load_dataset

data_files = {
    'train': '/content/train_data.json',
    'test': '/content/test_data.json',
    'val': '/content/val_data.json'


}

# Loading the dataset from the JSON files
dataset = load_dataset('json', data_files=data_files, split={'train': 'train', 'test': 'test', 'val': 'val'})


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

##**Exploratory Data Analysis (EDA)**

In [ ]:
from transformers import T5Tokenizer


import pandas as pd

movie_details = pd.read_json('/content/IMDB_movie_details.json', lines=True)

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def calculate_token_length(text):
    return len(tokenizer.tokenize(text))

movie_details['token_length'] = movie_details['plot_synopsis'].apply(calculate_token_length)

stats = movie_details['token_length'].describe()
print(stats)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


count     1572.000000
mean      2116.767812
std       2197.103130
min          0.000000
25%        729.000000
50%       1581.000000
75%       2814.750000
max      18103.000000
Name: token_length, dtype: float64


In [ ]:
# Apply the function to the plot_synopsis column
movie_details['token_length'] = movie_details['plot_summary'].apply(calculate_token_length)

# Display statistics about the token lengths
stats = movie_details['token_length'].describe()
print(stats)


count    1572.000000
mean      151.191476
std        60.718672
min        20.000000
25%       103.000000
50%       142.000000
75%       195.250000
max       315.000000
Name: token_length, dtype: float64


In [ ]:
device = 'cuda'


##**Preprocess the Data**

In [ ]:
import re
import torch
from transformers import LEDForConditionalGeneration, LEDTokenizer

# Load tokenizer and model
tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
model = LEDForConditionalGeneration.from_pretrained('allenai/led-base-16384')

model = model.to(device)

# Function to normalize text
def normalize_text(text):
    text = text.lower()  # Lowercase the text
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and newlines
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    return text

# Preprocess function with normalization
def preprocess_function(examples):
    # Normalize the plot_synopsis and plot_summary
    inputs = ["summarize: " + normalize_text(doc) for doc in examples["plot_synopsis"]]
    model_inputs = tokenizer(inputs, max_length=3000, truncation=True, padding="max_length", return_tensors="pt")

    # Normalize labels (plot_summary)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer([normalize_text(doc) for doc in examples["plot_summary"]], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

    # Replace -100 for padding tokens in labels
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in lab]
        for lab in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/236 [00:00<?, ? examples/s]

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

##**Define the Model**

In [ ]:
# from transformers import T5ForConditionalGeneration
import torch, gc

gc.collect()
torch.cuda.empty_cache()

# model = T5ForConditionalGeneration.from_pretrained('t5-small')


In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device:  {device}")

using device:  cuda


##**Train the LED Model**

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # LED requires a lot of memory
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    report_to="none"
)

# Initialize AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    tokenizer=tokenizer,
    optimizers=(optimizer, None) # Pass the optimizer to the Trainer
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.925300,2.873685


Epoch,Training Loss,Validation Loss
1,2.925300,2.873685
2,2.409600,2.883492
3,2.165300,2.920285


TrainOutput(global_step=3300, training_loss=2.429696747750947, metrics={'train_runtime': 5007.437, 'train_samples_per_second': 0.659, 'train_steps_per_second': 0.659, 'total_flos': 6526373990400000.0, 'train_loss': 2.429696747750947, 'epoch': 3.0})

##**Save the Model (3 epochs trained)**

In [ ]:
model_save_path = '/content/drive/MyDrive/summary_generation_Led_3'
trainer.save_model(model_save_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the trained model from Google Drive
model_save_path = '/content/drive/MyDrive/summary_generation_Led_3'
model = LEDForConditionalGeneration.from_pretrained(model_save_path)

model = model.to(device)


##**Continue training for 1 additional epoch**

In [ ]:
from transformers import Trainer, TrainingArguments

# Reload the model from the saved checkpoint in Google Drive
model_save_path = '/content/drive/MyDrive/summary_generation'  # Your saved path
model = LEDForConditionalGeneration.from_pretrained(model_save_path)
model = model.to(device)  # Ensure the model is on the right device (GPU or CPU)

# Define the training arguments for continuing training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # LED is memory-intensive
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,  # Only keep the last 3 models saved
    num_train_epochs=1,  # Continue training for 1 additional epoch
    report_to="none",
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,
    load_best_model_at_end=True,

# Initialize the optimizer (AdamW) again for the new training run
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    tokenizer=tokenizer,
    optimizers=(optimizer, None)
)

trainer.train()

trainer.save_model("/content/drive/MyDrive/summary_generation_Led_4")  # Save the new checkpoint


Epoch,Training Loss,Validation Loss
1,2.140500,2.964694


There were missing keys in the checkpoint model loaded: ['led.encoder.embed_tokens.weight', 'led.decoder.embed_tokens.weight', 'lm_head.weight'].


##**Evaluation on test data**

In [ ]:
pip install rouge_score

In [ ]:
pip install nltk


In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()


In [ ]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from datasets import load_metric
import nltk

metric_rouge = load_metric("rouge")

def generate_summary(batch):
    inputs = tokenizer(batch["plot_synopsis"], max_length=3000, truncation=True, padding="max_length", return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model.generate(inputs["input_ids"],  max_length=315, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)

    batch["pred_summary"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

results = dataset["test"].map(generate_summary, batched=True, batch_size=8)
rouge_score = metric_rouge.compute(predictions=results["pred_summary"], references=results["plot_summary"])
print("ROUGE scores:")
print(rouge_score)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

ROUGE scores:
{'rouge1': AggregateScore(low=Score(precision=0.2765937483910001, recall=0.28995076740292736, fmeasure=0.26035670057622456), mid=Score(precision=0.2921494324205486, recall=0.3074020206087751, fmeasure=0.2699305362452888), high=Score(precision=0.3059451759910771, recall=0.32368159175206357, fmeasure=0.27908123059966716)), 'rouge2': AggregateScore(low=Score(precision=0.05483977819595639, recall=0.05782616778822425, fmeasure=0.05115662969998852), mid=Score(precision=0.059651412474327724, recall=0.0629010316508109, fmeasure=0.05502744836975126), high=Score(precision=0.06481477377185653, recall=0.06911828260432601, fmeasure=0.05934541057407643)), 'rougeL': AggregateScore(low=Score(precision=0.17193896723048094, recall=0.17809953193399178, fmeasure=0.16095113303292502), mid=Score(precision=0.1815783962174589, recall=0.1874934529402342, fmeasure=0.16563763481163463), high=Score(precision=0.19066115263066233, recall=0.19614412850133803, fmeasure=0.1701542843468328)), 'rougeLsum':

In [ ]:
import nltk
from datasets import load_metric
nltk.download('punkt')  # Ensure the NLTK tokenizer is available

def calculate_bleu_scores(predictions, references):
    # Load the BLEU metric
    metric_bleu = load_metric("bleu")

    # Tokenize the predictions and references
    tokenized_predictions = [nltk.word_tokenize(pred) for pred in predictions]
    tokenized_references = [[nltk.word_tokenize(ref)] for ref in references]  # BLEU expects a list of list of references

    # Calculate BLEU scores
    bleu_score = metric_bleu.compute(predictions=tokenized_predictions, references=tokenized_references)

    return bleu_score

bleu_scores = calculate_bleu_scores(predictions=results["pred_summary"], references=results["plot_summary"])

print("BLEU scores:")
print(bleu_scores)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU scores:
{'bleu': 0.03487780615578816, 'precisions': [0.273911286562432, 0.05937059652418976, 0.016940597495189124, 0.005371388615198805], 'brevity_penalty': 1.0, 'length_ratio': 1.1596914314552467, 'translation_length': 32171, 'reference_length': 27741}


In [ ]:
import nltk
from datasets import load_metric
from prettytable import PrettyTable

print("BLEU scores:")
print(bleu_scores)

# Prepare a pretty table to display the BLEU scores
table = PrettyTable()
table.field_names = ["BLEU Score", "Value"]

# Assuming bleu_scores contains 'precisions' or equivalent
if 'precisions' in bleu_scores:
    for i, score in enumerate(bleu_scores['precisions'], start=1):
        table.add_row([f"BLEU-{i}", score])

# Print the pretty table
print("BLEU scores:")
print(table)


BLEU scores:
{'bleu': 0.03487780615578816, 'precisions': [0.273911286562432, 0.05937059652418976, 0.016940597495189124, 0.005371388615198805], 'brevity_penalty': 1.0, 'length_ratio': 1.1596914314552467, 'translation_length': 32171, 'reference_length': 27741}
BLEU scores:
+------------+----------------------+
| BLEU Score |        Value         |
+------------+----------------------+
|   BLEU-1   |  0.273911286562432   |
|   BLEU-2   | 0.05937059652418976  |
|   BLEU-3   | 0.016940597495189124 |
|   BLEU-4   | 0.005371388615198805 |
+------------+----------------------+


In [ ]:
from datasets import load_metric
from prettytable import PrettyTable

def print_rouge_table(rouge_scores):
    table = PrettyTable()
    table.field_names = ["Metric", "Confidence", "Precision", "Recall", "F-Measure"]

    # Iterate through each rouge type and its corresponding scores
    for rouge_type, aggregate_scores in rouge_scores.items():
        # Iterate through confidence levels: low, mid, high
        for confidence_level in ['low', 'mid', 'high']:
            score = getattr(aggregate_scores, confidence_level)
            table.add_row([
                rouge_type.upper(),
                confidence_level.capitalize(),
                f"{score.precision:.4f}",
                f"{score.recall:.4f}",
                f"{score.fmeasure:.4f}"
            ])

    # Print the table
    print(table)

# Call the function to print the table
print_rouge_table(rouge_score)

+-----------+------------+-----------+--------+-----------+
|   Metric  | Confidence | Precision | Recall | F-Measure |
+-----------+------------+-----------+--------+-----------+
|   ROUGE1  |    Low     |   0.2766  | 0.2900 |   0.2604  |
|   ROUGE1  |    Mid     |   0.2921  | 0.3074 |   0.2699  |
|   ROUGE1  |    High    |   0.3059  | 0.3237 |   0.2791  |
|   ROUGE2  |    Low     |   0.0548  | 0.0578 |   0.0512  |
|   ROUGE2  |    Mid     |   0.0597  | 0.0629 |   0.0550  |
|   ROUGE2  |    High    |   0.0648  | 0.0691 |   0.0593  |
|   ROUGEL  |    Low     |   0.1719  | 0.1781 |   0.1610  |
|   ROUGEL  |    Mid     |   0.1816  | 0.1875 |   0.1656  |
|   ROUGEL  |    High    |   0.1907  | 0.1961 |   0.1702  |
| ROUGELSUM |    Low     |   0.1724  | 0.1785 |   0.1610  |
| ROUGELSUM |    Mid     |   0.1817  | 0.1876 |   0.1658  |
| ROUGELSUM |    High    |   0.1910  | 0.1965 |   0.1706  |
+-----------+------------+-----------+--------+-----------+


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
